In [ ]:
# 抽象基类的常见用途：实现接口时作为超类使用。 Python中协议时接口，但不是正式的（只由文档和约定定义），因此协议不能像正式接口那样施加限制。
# 对象的类型无关紧要，只要实现了特定的协议即可。

# 自定义抽象基类，通常不需要这么做。
import abc

class Tombola(abc.ABC):  # 抽象基类要继承自abc.ABC (Abstract Base Class)
    # 从有限的集合中随机挑选物品的机器，直到选完为止
    @abc.abstractmethod
    def load(self, iterable):  # 抽象方法要使用装饰器abc.abstractmethod，内容为函数文档字符串
        """Add items from an iterable.""" # 把元素放入容器

    @abc.abstractmethod
    def pick(self):  # 随机选择物品
        """Remove item at random, returning it.

        This method should raise `LookupError` when the instance is empty.
        """

    def loaded(self):  # 具体方法 判断是否还有物品
        """Return `True` if there's at least 1 item, `False` otherwise."""
        return bool(self.inspect())  # 只能依赖抽象基类定义的接口（具体方法、抽象方法或特性）

    def inspect(self): # 具体方法 返回有序元组
        """Return a sorted tuple with the items currently inside."""
        items = []
        while True: # 不知道具体存储方式，用pick方法获取
            try: # 依赖pick在取完后抛出异常
                items.append(self.pick())
            except LookupError:
                break
        self.load(items)  # 将元素又放回
        return tuple(sorted(items)) # 返回有序元组

# 如果需要可以在@abc.abstractmethod装饰器上叠加@classmethod @property @staticmethod装饰器    
    
    
class Fake(Tombola):
    def pick(self):
        return 13
f = Fake() # 异常，load抽象方法未实现

In [ ]:

class BingoCage(Tombola):  # 子类

    def __init__(self, items):
        self._randomizer = random.SystemRandom()  # random.SystemRandom使用os.urandom
        self._items = []
        self.load(items)

    def load(self, items): # 实现抽象方法
        self._items.extend(items)
        self._randomizer.shuffle(self._items)  # 使用SystemRandom实例的shuffle方法

    def pick(self):  # 实现抽象方法
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')

    def __call__(self):  # 额外方法
        self.pick()


class LotteryBlower(Tombola): # 子类

    def __init__(self, iterable):
        self._balls = list(iterable)  # 以可迭代对象新构建list

    def load(self, iterable): # 实现抽象方法
        self._balls.extend(iterable)

    def pick(self): # 实现抽象方法
        try:
            position = random.randrange(len(self._balls))  # 如果为空则抛出异常
        except ValueError:
            raise LookupError('pick from empty BingoCage')
        return self._balls.pop(position)  # 使用pop保证元素不会被重复选中

    def loaded(self):  # 改写抽象基类方法 - 更高效
        return bool(self._balls)

    def inspect(self):  # 改写抽象基类方法 - 更高效
        return tuple(sorted(self._balls))

@Tombola.register  # 把TomboList注册为Tombola的虚拟子类
class TomboList(list):  # 继承自list

    def pick(self):
        if self:  # 从list中继承了__bool__方法
            position = randrange(len(self))
            return self.pop(position)  # 调用继承list的pop方法
        else:
            raise LookupError('pop from empty TomboList')

    load = list.extend  # 用继承list的extend方法实现Tombola的抽象方法load

    def loaded(self):
        return bool(self)  # 改写Tombola的方法

    def inspect(self): # 改写Tombola的方法
        return tuple(sorted(self))

# Tombola.register(TomboList)  # Python3.3之前，必须调用方法，而不能使用类装饰器

print(issubclass(TomboList, Tombola))
t = TomboList(range(100))
print(isinstance(t, Tombola))
print(TomboList.__mro__)   # mro即方法解析顺序 Method Resolution Order
# mro中只有真实超类list和object，没有虚拟超类
print(Tombola.__subclasses__()) # 获得类的直接子类列表，不含虚拟子类
print(list(Tombola._abc_registry)) # 只有抽象对象有这个属性，值为一个WeakSet对象，即抽象类注册的虚拟子类的弱引用
